In [ ]:
import numpy as np
import pandas as pd
from itertools import combinations
import random
import math

data = pd.read_csv(r'dados/movie_metadata.csv')
budgets = [0.1, 1, 10]

# não roda no meu note, tentei reduzir a quantidade de entradas só pra ver se tava gerando o CSV
def drop_null(data):
    data = data.drop_duplicates()
    # itera com duas flags em dataframes, pra índice e elemento
    for i, j in data.iterrows():
        if(math.isnan(j['gross']) or type(j['language']) == float):
            data = data.drop(i)
    return data[:1000]

data = drop_null(data)
data.head(10)

# Consultas

In [2]:
# Consulta 1: Filme com a maior arrecadação
def consulta1(data):
    aux = data[['movie_title', 'gross'][:]]
    return aux.loc[aux.gross.idxmax()]

# Consulta 2: Filme com maior arrecadação para cada idioma
def consulta2(data):
    aux = data[['movie_title', 'language', 'gross'][:]]
    return aux.loc[aux.groupby('language').gross.idxmax()]

# Consulta 3: Top 3 países com a maior quantidade de filmes
def consulta3(data):
    result = []
    aux = data[['country', 'movie_title'][:]]
    country_list = aux.groupby('country')['movie_title'].nunique().to_frame()
    country_list['country'] = country_list.index # o índice se perde ao agrupar
    country_list.reset_index(drop=True, inplace=True)
    for i in range(3):
        result.append(country_list.loc[country_list.movie_title.idxmax()])
        country_list = country_list.drop(country_list.movie_title.idxmax())
    return pd.DataFrame(result)

# Funções Score

In [3]:
def score1(item, out):
    score = 0
    if(not np.isnan(item.gross) and item.movie_title != 'None'):
        if(item.movie_title == out):
            score = item.gross
    return score

# o score da segunda consulta é igual ao da primeira

def score3(data, item, out):
    score = 0
    country_list = data.country.array
    countries = {}
    for i in range(0, len(country_list)):
        if(country_list[i] not in countries):
            countries[country_list[i]] = 0
    for i in range(0, len(data)):
        countries[data.iloc[i].country] += 1
    if(item.country == out):
        score = countries[item.country]
    return score

# Funções de sensibilidade

In [4]:
def sense_q1(data):
    mat = []
    sense = 0
    results = data[['movie_title'][:]]
    for i in range(0, len(results)):
        mat.append(score1(data.iloc[i], results.iloc[i].movie_title))
        
    for i in range(0, len(mat) - 1):
        sense = max([sense, abs(mat[i] - mat[i + 1])])
    return sense

# como os dois scores são iguais, a função de sensibilidade pode ser a mesma

def sense_q3(data):
    sense = 0
    country_list = []
    mat = []
    for i in range(0, len(data)):
        if(data.iloc[i].country not in country_list and data.iloc[i].country != 'None'):
            country_list.append(data.iloc[i].country)
    results = list(combinations(country_list, 3))
    
    for r in range(0, len(results)):
        mat.append(score3(data, data.iloc[r], results[r]))

    for i in range(0, len(mat) - 1):
        sense = max([sense, abs(mat[i] - mat[i + 1])])
    return sense

# Randomização de consultas

In [5]:
def rand_q1(budget, sense, data):
    outputs = data.movie_title
    probs = []
    pk = []
    k = np.arange(len(data))
    
    for i in outputs:
        row = []
        for j in range(0, len(data)):
            num = np.exp(budget * score1(data.iloc[j], i) / (2 * sense))
            den = 0
            for k in outputs:
                den += np.exp(budget * score1(data.iloc[j], k) / (2 * sense))
            row.append(num / den)
        i += 1
        probs.append(row)
        
    for i in probs:
        pk.append(np.sum(i) / np.sum(probs))
    return data[stats.rv_discrete(name='dist', values=(k, pk)).rvs(), 11]

    
def rand_q2(budget, sense, data):
    out = []
    lang = np.unique(data.language)
    clusters = len(lang) * [[]]
    for i in range(0, len(lang)):
        clang = []
        for j in range(0, len(data)):
            if(data.language == lang[i]):
                clang.append(data.iloc[j])
                clusters[i] = clang
    for i in clang:
        out.append(rand_q1(budget, sense, np.array(i)))
    return out


def rand_q3(budget, sense, data):
    budget /= 3
    probs = []
    pk = []
    rand = []
    out = np.unique(data.country)
    k = np.arange(len(out))
    
    for i in out:
        row = []
        for j in range(0, len(data)):
            num = np.exp(budget * score3(data.iloc[j], i) / (2 * sense))
            den = 0
            for k in out:
                den += np.exp(budget * score3(data.iloc[j], i) / (2 * sense))
            row.append(num / den)
        probs.append(row)
        
    for i in probs:
        pk.append(np.sum(prob) / np.sum(probs))

    for i in range(0, 3):
        is_valid = False
        while(not is_valid):
            reg = stats.rv_discrete(name='dist', values=(k, pk)).rvs()
            if(out[reg] not in rand):
                rand.append(out[reg])
                valido = True
    return rand

# Execução

In [ ]:
sense1 = sense_q1(data)

queries = [consulta1, consulta2, consulta3]
rand_queries = [rand_q1, rand_q2, rand_q3]
senses = [sense1, sense1, sense_q3(data)]

def dp_exponential(budgets, res, senses):
    res_data = []
    for i in range(0, len(budgets)):
        reg = [budgets[i]]
        for j in range(0, len(res[i])):
            reg.append(res[i][j])
        for k in range(0, len(senses)):
            reg.append(senses[k])
        res_data.append(reg)
    res_data = pd.DataFrame(res_data, columns=['budget', 'result_q1', 'result_q2', 'result_q3', 'sens_q1', 'sens_q2', 'sens_q3'])
    res_data.to_csv(r'saida/result.csv', index=False)
    print("Arquivo CSV gerado!")
    
def responses(data, senses, budgets, rand_list):
    result = []
    for i in budgets:
        res = []
        for j in range(0, len(rand_list)):
            q = rand_list[j](i, senses[j], data)
            res.append(q)
        result.append(res)
    return result
            
total_results = responses(data, senses, budgets, rand_queries)  

In [ ]:
dp_exponential(budgets, total_results, senses)